In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import time
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq
import wandb
plt.switch_backend('agg')
import matplotlib.ticker as ticker

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 30

In [3]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {}
        self.char2count = {}
        self.index2char = {0: "SOS", 1: "EOS"}
        self.n_chars = 2  # Count SOS and EOS

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1

In [4]:
def readLangs(lang, type, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    
    lines = open('aksharantar_sampled/%s/%s_%s.csv' % (lang, lang, type), encoding='utf-8').read().strip().split('\n')
    
    # Split every line into pairs and normalize
    # pairs = [[normalizeString(s) for s in l.split(',')] for l in lines]
    pairs = [[s for s in l.split(',')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang)
        output_lang = Lang('eng')
    else:
        input_lang = Lang('eng')
        output_lang = Lang(lang)

    return input_lang, output_lang, pairs

In [5]:
def prepareData(lang, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang, reverse)
    print("Read %s word pairs" % len(pairs))
    print("Trimmed to %s word pairs" % len(pairs))
    print("Counting chars...")
    for pair in pairs:
        input_lang.addWord(pair[0])
        output_lang.addWord(pair[1])
    print("Counted chars:")
    print(input_lang.name, input_lang.n_chars)
    print(output_lang.name, output_lang.n_chars)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('mni', 'train')
print(random.choice(pairs))

Reading lines...
Read 10060 word pairs
Trimmed to 10060 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
['indrabu', 'ꯏꯟꯗ꯭ꯔꯕꯨ']


In [6]:
cells = {'rnn': nn.RNN, 'gru': nn.GRU, 'lstm': nn.LSTM}

In [7]:
class EncoderRNN(nn.Module):
    """
    Class for the encoder RNN.
    """
    def __init__(self, input_size, embedding_size, hidden_size, num_hidden_layers, dropout=0.2, cell_type='gru'):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)

    def forward(self, input, hidden, cell=None):
        embedded = self.embedding(input).view(1, 1, -1)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(embedded, hidden)
            return output, hidden
        else:
            output, (hidden, cell) = self.cell(embedded, (hidden, cell))    
            return output, hidden, cell

    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

In [8]:
class BeamSearchNode:
    def __init__(self, decoder_output, hidden, prev_node, char_idx, log_prob, length):
        self.decoder_output = decoder_output
        self.hidden = hidden
        self.prev_node = prev_node
        self.char_idx = char_idx
        self.log_prob = log_prob
        self.length = length

    # def __lt__(self, other):
    #     return self.log_prob > other.log_prob

    def eval(self):
        return self.log_prob / self.length

In [9]:
class DecoderRNN(nn.Module):
    """
    Class for the vanilla decoder RNN.
    """
    def __init__(self, hidden_size, embedding_size, output_size, num_hidden_layers, dropout=0, cell_type='gru'):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_hidden_layers = num_hidden_layers
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.cell_type = cell_type
        self.cell = cells[cell_type](embedding_size, hidden_size, num_hidden_layers, dropout=dropout)
        self.out = nn.Linear(hidden_size, output_size)
        self.output_size = output_size
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden, cell=None):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            output, hidden = self.cell(output, hidden)
        else:
            output, (hidden, cell) = self.cell(output, (hidden, cell))
            
        output = self.softmax(self.out(output[0]))

        if self.cell_type == 'rnn' or self.cell_type == 'gru':
            return output, hidden
        else:
            return output, hidden, cell
    
    def initHidden(self):
        return torch.zeros(self.num_hidden_layers, 1, self.hidden_size, device=device)

    def beam_search(self, encoder_outputs, decoder_hidden, beam_width, max_length, start_token, end_token):
        # Initialize the beam search
        beam_nodes = [BeamSearchNode(None, decoder_hidden, None, start_token, 0, 1)]
        done_nodes = []

        # Keep expanding the beam until we reach the maximum length or all candidates are done
        for _ in range(max_length):
            candidates = []
            for node in beam_nodes:
                if node.char_idx == end_token:
                    done_nodes.append(node)
                    continue

                # Feed the previous char and hidden state into the decoder
                output, hidden = self.forward(torch.tensor([node.char_idx]), node.hidden)
                # Single depth or double depth for tensor?

                # Generate new candidate nodes and add them to the heap
                for char_idx in range(self.output_size):
                    length = node.length + 1
                    log_prob = node.log_prob + output[0][char_idx].item()
                    new_node = BeamSearchNode(output, hidden, node, char_idx, log_prob, length)
                    heapq.heappush(candidates, (-new_node.eval(), id(new_node), new_node))

            # Select the top k candidates to continue expanding the beam
            beam_nodes = []
            for _ in range(beam_width):
                if not candidates:
                    break
                _, _, node = heapq.heappop(candidates)
                beam_nodes.append(node)

            if not beam_nodes:
                break

        # Return the best candidate as the predicted sequence
        done_nodes.extend(beam_nodes)
        best_node = max(done_nodes, key=lambda node: node.eval())
        # predicted_seq = []
        # while best_node.prev_node is not None:
        #     predicted_seq.append(best_node.char_idx)
        #     best_node = best_node.prev_node
        # return predicted_seq[::-1]
        return best_node

In [10]:
def indexesFromWord(lang, word):
    return [lang.char2index[char] for char in word]


def tensorFromWord(lang, word):
    indexes = indexesFromWord(lang, word)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromWord(input_lang, pair[0])
    target_tensor = tensorFromWord(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [11]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    if encoder.cell_type == 'lstm':
        encoder_cell = encoder.initHidden()

    loss = 0

    for ei in range(input_length):
        # print(cell_type)
        if encoder.cell_type == 'lstm':
            encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
        else:
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_hidden = encoder_hidden

    if beam_width == 1:
        decoder_input = torch.tensor([[SOS_token]], device=device)

        if decoder.cell_type == 'lstm':
            decoder_cell = encoder_cell

        use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                decoder_input = target_tensor[di]  # Teacher forcing

        else:
            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze().detach()  # detach from history as input

                # print("Criterion:", criterion(decoder_output, target_tensor[di])) 
                loss += criterion(decoder_output, target_tensor[di])
                # print(loss)
                if decoder_input.item() == EOS_token:
                    break

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length
    
    else:
        best_node = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, target_length, SOS_token, EOS_token)

        decoder_outputs = []
        curr_node = best_node

        while curr_node.decoder_output is not None:
            decoder_outputs.append(curr_node.decoder_output)
            curr_node = curr_node.prev_node

        decoder_outputs = decoder_outputs[::-1]

        # Loss is the sum of the criterion of each decoder output with respect to the target
        loss = sum([criterion(decoder_output, target_tensor[i]) for i, decoder_output in enumerate(decoder_outputs)])
        # print(loss)
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length


In [12]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [13]:
def trainIters(encoder, decoder, n_iters, optimizer=optim.SGD, cell_type='gru', beam_width=1, print_every=1000, plot_every=100, learning_rate=5e-3):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for _ in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, cell_type, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

In [14]:
def evaluate(encoder, decoder, word, beam_width=1, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromWord(input_lang, word)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        if encoder.cell_type == 'lstm':
            encoder_cell = encoder.initHidden()

        for ei in range(input_length):
            if encoder.cell_type == 'lstm':
                encoder_output, encoder_hidden, encoder_cell = encoder(input_tensor[ei], encoder_hidden, encoder_cell)
            else:
                encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
                
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_hidden = encoder_hidden

        if beam_width == 1:
            decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

            if encoder.cell_type == 'lstm':
                decoder_cell = encoder_cell

            decoded_chars = ""
            # decoder_attentions = torch.zeros(max_length, max_length)

            for di in range(max_length):
                # decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
                if decoder.cell_type == 'lstm':
                    decoder_output, decoder_hidden, decoder_cell = decoder(decoder_input, decoder_hidden, decoder_cell)
                else:
                    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
                    
                # decoder_attentions[di] = decoder_attention.data
                topv, topi = decoder_output.data.topk(1)
                if topi.item() == EOS_token:
                    decoded_chars += '<EOS>'
                    break
                else:
                    decoded_chars += output_lang.index2char[topi.item()]

                decoder_input = topi.squeeze().detach()

            # return decoded_chars, decoder_attentions[:di + 1]
            return decoded_chars
        
        else:
            predicted_seq = decoder.beam_search(encoder_outputs, decoder_hidden, beam_width, SOS_token, EOS_token)
            return ''.join([output_lang.index2char[char_idx] for char_idx in predicted_seq])

In [15]:
input_valid_lang, output_valid_lang, valid_pairs = prepareData('mni', 'valid')
input_test_lang, output_test_lang, test_pairs = prepareData('mni', 'test')

def evalAccuracy(encoder, decoder, pairs):
    correct = 0
    for pair in pairs:
        output_word = evaluate(encoder, decoder, pair[0])
        output_word = output_word[:-5]
        if output_word == pair[1]:
            correct += 1
    return correct / len(pairs)

Reading lines...
Read 3260 word pairs
Trimmed to 3260 word pairs
Counting chars...
Counted chars:
eng 28
mni 46
Reading lines...
Read 4096 word pairs
Trimmed to 4096 word pairs
Counting chars...
Counted chars:
eng 28
mni 46


In [16]:
def trainBatch(encoder, decoder, epochs, optimizer=optim.SGD, beam_width=1, print_every=1000, learning_rate=5e-3, log=False):
    start = time.time()
    print_loss_total = 0  # Reset every print_every

    encoder_optimizer = optimizer(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optimizer(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(pair) for pair in pairs]
    criterion = nn.NLLLoss()

    # print("Training pairs:", len(training_pairs))

    for iter in range(1, len(training_pairs) * epochs + 1):
        training_pair = training_pairs[(iter - 1) % len(training_pairs)]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, beam_width)
        print_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / (len(training_pairs) * epochs)), iter, iter / (len(training_pairs) * epochs) * 100, print_loss_avg))

        if log:
            if iter % print_every == 0:
                wandb.log({"train_loss": loss})

            if iter % len(training_pairs) == 0:
                wandb.log({"train_accuracy": evalAccuracy(encoder, decoder, pairs), "val_accuracy": evalAccuracy(encoder, decoder, valid_pairs), "epoch": iter / len(training_pairs)})

In [17]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        # output_chars, attentions = evaluate(encoder, decoder, pair[0])
        output_word = evaluate(encoder, decoder, pair[0])
        # output_word = ''.join(output_chars)
        print('<', output_word)
        print('')

In [1253]:
hidden_size = 256
# embedding_size = input_lang.n_chars
embedding_size = 256
encoder1 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=1, cell_type='gru').to(device)
decoder1 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=1, cell_type='gru').to(device)

In [1254]:
# trainIters(encoder1, decoder1, n_iters=1, beam_width=5, print_every=100)

In [1255]:
trainBatch(encoder1, decoder1, epochs=4, beam_width=1, print_every=100)

0m 1s (- 9m 55s) (100 0%) 2.6452
0m 2s (- 9m 39s) (200 0%) 2.7250
0m 4s (- 9m 29s) (300 0%) 2.8955
0m 5s (- 9m 42s) (400 0%) 2.6631
0m 7s (- 9m 30s) (500 1%) 2.7041
0m 8s (- 9m 28s) (600 1%) 2.6729
0m 9s (- 9m 16s) (700 1%) 2.6407
0m 11s (- 9m 5s) (800 1%) 2.6048
0m 12s (- 8m 58s) (900 2%) 2.6655
0m 13s (- 8m 50s) (1000 2%) 2.6220
0m 15s (- 9m 4s) (1100 2%) 2.7234
0m 17s (- 9m 17s) (1200 2%) 2.6269
0m 18s (- 9m 26s) (1300 3%) 2.6035
0m 20s (- 9m 42s) (1400 3%) 2.6171
0m 22s (- 9m 53s) (1500 3%) 2.5768
0m 24s (- 9m 43s) (1600 3%) 2.5544
0m 25s (- 9m 32s) (1700 4%) 2.5816
0m 26s (- 9m 21s) (1800 4%) 2.6929
0m 27s (- 9m 12s) (1900 4%) 2.6096
0m 28s (- 9m 5s) (2000 4%) 2.5607
0m 29s (- 8m 59s) (2100 5%) 2.6524
0m 30s (- 8m 53s) (2200 5%) 2.5363
0m 32s (- 8m 48s) (2300 5%) 2.6139
0m 33s (- 8m 43s) (2400 5%) 2.5911
0m 34s (- 8m 38s) (2500 6%) 2.6466
0m 35s (- 8m 32s) (2600 6%) 2.6185
0m 36s (- 8m 28s) (2700 6%) 2.5745
0m 37s (- 8m 23s) (2800 6%) 2.5042
0m 38s (- 8m 18s) (2900 7%) 2.5636
0m 3

In [1256]:
encoder2 = EncoderRNN(input_lang.n_chars, embedding_size, hidden_size, num_hidden_layers=2, cell_type='gru').to(device)
decoder2 = DecoderRNN(hidden_size, embedding_size, output_lang.n_chars, num_hidden_layers=2, cell_type='gru').to(device)

In [1257]:
trainBatch(encoder2, decoder2, epochs=4, beam_width=1, print_every=100)

0m 2s (- 16m 10s) (100 0%) 2.7145
0m 5s (- 17m 32s) (200 0%) 2.7649
0m 7s (- 17m 31s) (300 0%) 2.9577
0m 10s (- 17m 32s) (400 0%) 2.8226
0m 12s (- 17m 8s) (500 1%) 2.8124
0m 15s (- 16m 36s) (600 1%) 2.7360
0m 17s (- 16m 19s) (700 1%) 2.7871
0m 19s (- 16m 20s) (800 1%) 2.7793
0m 22s (- 16m 18s) (900 2%) 2.8150
0m 25s (- 16m 22s) (1000 2%) 2.8402
0m 27s (- 16m 33s) (1100 2%) 2.7464
0m 32s (- 17m 31s) (1200 2%) 2.6716
0m 36s (- 18m 1s) (1300 3%) 2.7219
0m 40s (- 18m 53s) (1400 3%) 2.6794
0m 44s (- 19m 16s) (1500 3%) 2.5827
0m 48s (- 19m 30s) (1600 3%) 2.6698
0m 53s (- 20m 9s) (1700 4%) 2.6470
0m 57s (- 20m 31s) (1800 4%) 2.6580
1m 1s (- 20m 45s) (1900 4%) 2.6411
1m 5s (- 20m 55s) (2000 4%) 2.6749
1m 10s (- 21m 12s) (2100 5%) 2.6077
1m 14s (- 21m 30s) (2200 5%) 2.6726
1m 19s (- 21m 45s) (2300 5%) 2.5735
1m 23s (- 21m 57s) (2400 5%) 2.5459
1m 28s (- 22m 9s) (2500 6%) 2.6169
1m 30s (- 21m 55s) (2600 6%) 2.6146
1m 35s (- 22m 10s) (2700 6%) 2.6166
1m 40s (- 22m 19s) (2800 6%) 2.5944
1m 44s (- 

In [1258]:
evaluateRandomly(encoder1, decoder1)

> pei
= ꯄꯩ
< ꯄꯩ<EOS>

> yourakpada
= ꯌꯧꯔꯛꯄꯗ
< ꯌꯧꯔꯛꯄꯗ<EOS>

> khangnaribashi
= ꯈꯪꯅꯔꯤꯕꯁꯤ
< ꯈꯪꯍꯔꯥꯡꯅꯤꯁ<EOS>

> laagumna
= ꯂꯥꯒꯨꯝꯅ
< ꯂꯥꯒꯒꯝꯅ<EOS>

> sabitri
= ꯁꯥꯕꯤꯇ꯭ꯔꯤ
< ꯁꯥꯢꯕ꯭ꯔꯤ<EOS>

> kakchinglakpa
= ꯀꯛꯆꯤꯡꯂꯥꯛꯄ
< ꯀꯥꯢꯆꯪꯂꯛꯄꯄ<EOS>

> waaba
= ꯋꯥꯕ
< ꯋꯥꯕ<EOS>

> soitapa
= ꯁꯣꯢꯇꯄ
< ꯁꯤꯢꯇꯄ<EOS>

> apaamba
= ꯑꯄꯥꯝꯕ
< ꯑꯄꯥꯝꯕ<EOS>

> iriba
= ꯏꯔꯤꯕ
< ꯏꯔꯤꯕ<EOS>



In [1142]:
# pair = random.choice(pairs)

# print(pair[0])
# print(pair[1])

# with torch.no_grad():
#     input_tensor = tensorFromWord(input_lang, pair[0])
#     input_length = input_tensor.size()[0]
#     encoder1_hidden = encoder1.initHidden()

#     encoder1_outputs = torch.zeros(MAX_LENGTH, encoder1.hidden_size, device=device)

#     if encoder1.cell_type == 'lstm':
#         encoder1_cell = encoder1.initHidden()

#     for ei in range(input_length):
#         if encoder1.cell_type == 'lstm':
#             encoder1_output, encoder1_hidden, encoder1_cell = encoder1(input_tensor[ei], encoder1_hidden, encoder1_cell)
#         else:
#             encoder1_output, encoder1_hidden = encoder1(input_tensor[ei], encoder1_hidden)
            
#         encoder1_outputs[ei] += encoder1_output[0, 0]

#     decoder1_hidden = encoder1_hidden

#     if encoder1.cell_type == 'lstm':
#         decoder1_cell = encoder1_cell

#     decoded_chars = ""

#     # Predict the output using beam search
#     predicted_seq = decoder1.beam_search(encoder1_outputs, decoder1_hidden, 5, SOS_token, EOS_token)

#     # Convert the predicted sequence to a word
#     for char_idx in predicted_seq:
#         decoded_chars += output_lang.index2char[char_idx]

#     print(decoded_chars)

    



In [1266]:
evalAccuracy(encoder2, decoder2, valid_pairs)

0.55

In [18]:
# sweep_configuration = {
#     'method': 'bayes',
#     'name': 'bayes-sweep',
#     'metric': {
#         'name': 'val_accuracy',
#         'goal': 'maximize'
#     },
#     'parameters': {
#         'epochs': {
#             'values': [5, 10] 
#         },
#         'embedding_size': {
#             'values': [64, 128, 256] 
#         },
#         'num_hidden_layers': {
#             'values': [1, 2, 3]
#         },
#         'hidden_layer_size': {
#             'values': [64, 128, 256] 
#         },
#         'cell_type': {
#             'values': ['rnn', 'gru', 'lstm'] 
#         },
#         'learning_rate': {
#             'values': [1e-2, 5e-3, 1e-3, 5e-4] 
#         },
#         'optimizer': {
#             'values': ['SGD', 'Adam'] 
#         },
#         'dropout': {
#             'values': [0.0, 0.2, 0.3] 
#         },
#         'beam_width': {
#             'values': [1] #[1, 3, 5]
#         }
#     }
# }


In [24]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'bayes-sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [10] 
        },
        'embedding_size': {
            'values': [64] 
        },
        'num_hidden_layers': {
            'values': [1]
        },
        'hidden_layer_size': {
            'values': [256] 
        },
        'cell_type': {
            'values': ['lstm'] 
        },
        'learning_rate': {
            'values': [1e-3] 
        },
        'optimizer': {
            'values': ['Adam'] 
        },
        'dropout': {
            'values': [0.2] 
        },
        'beam_width': {
            'values': [1] 
        }
    }
}


In [25]:
def hyperparameter_search():
    run = wandb.init(project='hyperparam-search-enc-dec', reinit=True)
    config = wandb.config
    wandb.run.name = f'hl_{config.num_hidden_layers}_sz_{config.hidden_layer_size}_ct_{config.cell_type}_lr_{config.learning_rate}_opt_{config.optimizer}_do_{config.dropout}_bw_{config.beam_width}_em_{config.embedding_size}'
    
    encoder = EncoderRNN(input_lang.n_chars, config.embedding_size, config.hidden_layer_size, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)
    decoder = DecoderRNN(config.hidden_layer_size, config.embedding_size, output_lang.n_chars, config.num_hidden_layers, dropout=config.dropout, cell_type=config.cell_type).to(device)

    if config.optimizer == 'SGD':
        optimizer = optim.SGD
    elif config.optimizer == 'Adam':
        optimizer = optim.Adam

    trainBatch(encoder, decoder, config.epochs, optimizer=optimizer, beam_width=config.beam_width, print_every=100, learning_rate=config.learning_rate, log=True)
    run.finish()

In [26]:
sweep_id = wandb.sweep(sweep_configuration, project='hyperparam-search-seq2seq')
wandb.agent(sweep_id, hyperparameter_search, count=2)

Create sweep with ID: 7vnrbrwr
Sweep URL: https://wandb.ai/abdullah_010/hyperparam-search-seq2seq/sweeps/7vnrbrwr


wandb: Agent Starting Run: 8cgrmprv with config:
wandb: 	beam_width: 1
wandb: 	cell_type: lstm
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: Adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/home/abdullah/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


0m 1s (- 26m 25s) (100 0%) 2.8445
0m 3s (- 26m 51s) (200 0%) 2.9171
0m 5s (- 27m 52s) (300 0%) 2.7625
0m 6s (- 27m 31s) (400 0%) 2.6095
0m 8s (- 28m 17s) (500 0%) 2.7432
0m 10s (- 30m 22s) (600 0%) 2.6398
0m 13s (- 31m 39s) (700 0%) 2.6101
0m 15s (- 32m 28s) (800 0%) 2.6033
0m 18s (- 34m 21s) (900 0%) 2.5363
0m 20s (- 33m 53s) (1000 0%) 2.5024
0m 22s (- 33m 23s) (1100 1%) 2.4859
0m 23s (- 33m 0s) (1200 1%) 2.4426
0m 26s (- 33m 17s) (1300 1%) 2.3194
0m 28s (- 33m 32s) (1400 1%) 2.1676
0m 30s (- 33m 56s) (1500 1%) 2.0778
0m 32s (- 33m 31s) (1600 1%) 2.1841
0m 34s (- 33m 1s) (1700 1%) 2.0148
0m 35s (- 32m 40s) (1800 1%) 2.0796
0m 37s (- 32m 24s) (1900 1%) 2.0305
0m 39s (- 32m 13s) (2000 1%) 1.8937
0m 40s (- 31m 55s) (2100 2%) 1.8485
0m 42s (- 31m 49s) (2200 2%) 1.7464
0m 44s (- 31m 46s) (2300 2%) 1.6868
0m 46s (- 31m 52s) (2400 2%) 1.6214
0m 48s (- 31m 59s) (2500 2%) 1.6652
0m 50s (- 31m 55s) (2600 2%) 1.5045
0m 52s (- 31m 56s) (2700 2%) 1.6071
0m 54s (- 31m 53s) (2800 2%) 1.4300
0m 57s (

epoch,▁▂▃▃▄▅▆▆▇█
train_accuracy,▁▄▅▆▇▇▇███
train_loss,▇▃▃█▇▁▁▁▁▂▁▂▂▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▂▁▁▁▁▄▁▁
val_accuracy,▁▅▆▇▇▇████
epoch,10.0
train_accuracy,0.86203
train_loss,0.0077
val_accuracy,0.65521


wandb: Agent Starting Run: o4d2veck with config:
wandb: 	beam_width: 1
wandb: 	cell_type: lstm
wandb: 	dropout: 0.2
wandb: 	embedding_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	learning_rate: 0.001
wandb: 	num_hidden_layers: 1
wandb: 	optimizer: Adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/home/abdullah/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


0m 2s (- 37m 23s) (100 0%) 2.8068
0m 4s (- 35m 5s) (200 0%) 2.8005
0m 6s (- 38m 43s) (300 0%) 2.8174
0m 10s (- 45m 39s) (400 0%) 2.6801
0m 13s (- 45m 36s) (500 0%) 2.6598
0m 15s (- 43m 58s) (600 0%) 2.6972
0m 17s (- 42m 35s) (700 0%) 2.5894
0m 19s (- 40m 55s) (800 0%) 2.4627
0m 21s (- 40m 2s) (900 0%) 2.5540
0m 23s (- 39m 34s) (1000 0%) 2.4733
0m 26s (- 39m 55s) (1100 1%) 2.4783
0m 28s (- 39m 40s) (1200 1%) 2.3469
0m 30s (- 39m 20s) (1300 1%) 2.3475
0m 33s (- 39m 2s) (1400 1%) 2.2316
0m 35s (- 38m 32s) (1500 1%) 2.0147
0m 37s (- 38m 10s) (1600 1%) 2.1142
0m 38s (- 37m 41s) (1700 1%) 2.0139
0m 40s (- 37m 18s) (1800 1%) 2.0074
0m 42s (- 36m 55s) (1900 1%) 1.9759
0m 44s (- 36m 29s) (2000 1%) 1.8694
0m 46s (- 36m 7s) (2100 2%) 1.8507
0m 48s (- 35m 51s) (2200 2%) 1.7019
0m 49s (- 35m 35s) (2300 2%) 1.6215
0m 51s (- 35m 17s) (2400 2%) 1.5938
0m 53s (- 35m 2s) (2500 2%) 1.6544
0m 55s (- 34m 47s) (2600 2%) 1.3870
0m 57s (- 34m 36s) (2700 2%) 1.5296
0m 59s (- 34m 22s) (2800 2%) 1.2993
1m 0s (- 

wandb: Ctrl + C detected. Stopping sweep.


1m 15s (- 33m 58s) (3600 3%) 1.2936
1m 17s (- 33m 48s) (3700 3%) 1.1299
